# Object Retrieval API
https://v2.sherpa.ac.uk/api/object-retrieval.html

In [3]:
# APIキーを.env に格納
from dotenv import load_dotenv
load_dotenv()
api_key = os.environ['api_key']
# print(api_key)
# data = [] # 初期化
data = list(range(7))  

In [5]:
# sherpaのAPIにクエリを投げる
import requests
import json
import pprint
# https://v2.sherpa.ac.uk/cgi/retrieve?item-type=repository&api-key=<YOUR-API-KEY>&format=Json&filter=[[%22country%22,%22equals%22,%22jp%22]]
url = 'https://v2.sherpa.ac.uk/cgi/retrieve'
filterparam = '[[%22country%22,%22equals%22,%22jp%22]]'
# apiの制限は1回100件なので、offsetで調整
params = {'item-type':'repository','api-key':api_key,'format':'Json','filter':filterparam, 'offset':'100'}
# python requests クエリストリングを encodeしないようにする
params_str = "&".join("%s=%s" % (k,v) for k,v in params.items())
res = requests.get(url, params=params_str)
# print(res.url)
data[1] = json.loads(res.text)
# pprint.pprint(data)
# about 6.4s/100ken

In [42]:
len(data)
concat_data = {}
# 上書きされる
#concat_data.update(data[0])
#concat_data.update(data[1]) 
#concat_data = {**data[0],**data[1]}
# for mylist in range(len(data)):
#    print(mylist)
#    #concat_data.update(data[mylist])
concat_data = data[0]['items'] + data[1]['items']

In [46]:
concat_data[199]

{'repository_metadata': {'oai_url': 'http://mu.repo.nii.ac.jp/oai',
  'content_types': ['journal_articles', 'theses_and_dissertations'],
  'type': 'institutional',
  'full_text_record_count': 1229,
  'content_subjects_phrases': [{'language': 'en',
    'phrase': 'Science',
    'value': 'science'},
   {'language': 'en', 'value': 'technology', 'phrase': 'Technology'},
   {'language': 'en', 'phrase': 'Engineering', 'value': 'engineering'},
   {'language': 'en', 'value': 'mathematics', 'phrase': 'Mathematics'},
   {'language': 'en',
    'phrase': 'Health and Medicine',
    'value': 'health_and_medicine'},
   {'language': 'en', 'value': 'arts', 'phrase': 'Arts'},
   {'phrase': 'Humanities', 'value': 'humanities', 'language': 'en'},
   {'phrase': 'Social Sciences',
    'value': 'social_sciences',
    'language': 'en'}],
  'software': {'name_phrases': [{'language': 'en',
     'value': 'weko',
     'phrase': 'WEKO'}],
   'name': 'weko'},
  'url': 'https://mu.repo.nii.ac.jp/',
  'content_subject

In [47]:
#import json 
#json_object = json.load(data)
#print(json_object["items"][0])
# すでに一つ上で json.load しているので、さらにloadしなくてよい（当たり前ですが・・・）
# data["items"][1] の　[0],[1],... がそれぞれ1機関
# content = concat_data["items"][99]
content = concat_data[99]
print(content)

{'repository_metadata': {'content_types_phrases': [{'language': 'en', 'phrase': 'Journal Articles', 'value': 'journal_articles'}, {'language': 'en', 'value': 'unpub_reports_and_working_papers', 'phrase': 'Reports and Working Papers'}, {'language': 'en', 'value': 'other_special_item_types', 'phrase': 'Other Special Item Types'}], 'type_phrases': [{'language': 'en', 'phrase': 'Institutional', 'value': 'institutional'}], 'metadata_record_count': 8433, 'content_subjects': ['social_sciences'], 'name': [{'language': 'en', 'name': 'Hokkaido University of Education Repository', 'preferred_phrases': [{'language': 'en', 'phrase': 'Name', 'value': 'name'}], 'language_phrases': [{'language': 'en', 'value': 'en', 'phrase': 'English'}], 'preferred': 'name'}, {'language': 'ja', 'preferred': 'name', 'language_phrases': [{'language': 'en', 'phrase': 'Japanese', 'value': 'ja'}], 'preferred_phrases': [{'value': 'name', 'phrase': 'Name', 'language': 'en'}], 'name': '北海道教育大学学術リポジトリ'}], 'url': 'http://s-ir.

In [50]:
len(concat_data)

200

In [53]:
def flatten(d,sep="_"):
    import collections
    obj = collections.OrderedDict()
    def recurse(t,parent_key=""):
        if isinstance(t,list):
            for i in range(len(t)):
                recurse(t[i],parent_key + sep + str(i) if parent_key else str(i))
        elif isinstance(t,dict):
            for k,v in t.items():
                recurse(v,parent_key + sep + k if parent_key else k)
        else:
            obj[parent_key] = t
    recurse(d)
    return obj

# data2 = flatten(data['items'])
data2 = []
for org in range(len(concat_data)):
    data2[org] = flatten(concat_data[org])  
# for org in range(len(data['items'])):
#     data2[org] = flatten(data['items'][org])  

IndexError: list assignment index out of range

In [ ]:
# print(data2.keys())
data2

In [ ]:
# Pandas Dataframe でキーを列に、値を行の値に変換するには、辞書を角括弧で囲むだけ
# 完全に一列フラットになってしまう
import pandas as pd
# repometa3 = pd.DataFrame([data2[0]])
# len(data2)
repometa3 = pd.DataFrame(index=[], columns=[])
for org in range(len(data['items'])):
    repometa3 = pd.concat([repometa3, pd.DataFrame([data2[org]])])
repometa3

In [ ]:
repometa3.set_index('system_metadata_id', drop=False, inplace=True)

In [ ]:
repometa3
# repometa3.columns
# repometa3['organisation_name_1_name']
# print(repometa3.iloc[97:,])

In [ ]:
import pandas as pd
repometa = pd.DataFrame(list(content.items())).T
# repometa = pd.DataFrame.from_dict(content, orient='index').T
repometa

In [ ]:
# content2 = data["items"]
# print(content2)
repometa2 = pd.DataFrame(data["items"])
repometa2

In [ ]:
repometa2

In [ ]:
repometa3 = repometa2.copy()

In [ ]:
# pandasのセルのなかにdictionaryが！を展開
# repometa4 = repometa2['repository_metadata'].apply(pd.Series)
# これだけだとさらにdarrayが埋まっている
'''
for i, column_name in enumerate(repometa3):
   repometa4 = repometa3[column_name].apply(pd.Series)
'''

def flatdict():
   for i, column_name in enumerate(repometa3):
      repometa4 = repometa3[column_name].apply(pd.Series)
   return repometa4

repometa4 = flatdict()


In [ ]:
# data['items'][0]
repometa4

In [ ]:
#v for i, column_name in enumerate(repometa4):
#    print(column_name)
#for i, column_name in enumerate(repometa4):
column_name = 'content_types_phrases'
repometa5 = repometa4[column_name].apply(pd.Series)

In [ ]:
repometa5

In [ ]:
def flatdict():
   for i, column_name in enumerate(repometa4):
      repometa5 = repometa4[column_name].apply(pd.Series)
   return repometa5
flatdict()

In [ ]:
repometa4

In [ ]:

# for i, column_name in enumerate(repometa3):
column_name = 'software'
df = pd.DataFrame(repometa4[column_name].values.tolist()).add_prefix(column_name)
# df = pd.DataFrame(repometa4['content_types'].values.tolist()).add_prefix('content_types_')
# df = pd.DataFrame(repometa2['type'].values.tolist()).add_prefix('content_types_')
'''
def flatarray():
   for i, column_name in enumerate(repometa2):
       pd.DataFrame(repometa2[column_name].values.tolist()).add_prefix(column_name)
       # df2 = pd.DataFrame(repometa2[columun_name].values.tolist()).add_prefix(column_name)
       # repometa3 = pd.concat(repometa2, df2)
       # repometa2.assign(df2)
       # repometa2[column_name].apply(pd.DataFrame(repometa2[columun_name].values.tolist()))
       # repometa2[column_name].apply(pd.Series)
   return flatarray()

df2 = flatarray()
'''



In [ ]:
df

In [ ]:
def _unwrap(dct, prefix):
    for key, value in dct.items():
        new_key = f'{prefix}_{key}'
        if isinstance(value, dict):
            yield from _unwrap(value, new_key)
        else:
            yield new_key[1:], value       

def unwrap(dct):
    return dict(_unwrap(dct, ''))

content2 = repometa2['name']
d2 = unwrap(content2)
d2


In [ ]:
repometa2 = pd.DataFrame(data["items"])
repometa2